In [25]:
# Библиотеки

from scipy.integrate import solve_ivp
import numpy as np
import matplotlib.pyplot as plt

import torch 
import torch.optim as optim
from torchdiffeq import odeint             # пока не понятно для чего

In [ ]:
# Чтобы при вызове print или display после точки выводилось фиксированное число знаков

np.set_printoptions(precision=5, suppress=True)

In [27]:
# Переменные

IS_RESTART = False
P_CUTOFF = 0.0                                                     # Непонятно для чего
N_EPOCH = 1000                                                     # Количество эпох. Максимальное
N_PLOT = 100                                                       # Частота формирования графиков. Через сколько эпох

# Есть большая вероятность, что нам не нужен будет OPT, и что строчку ниже нужно будет удалить
OPT = optim.AdamW([torch.tensor(1.0)], lr=0.001, betas=(0.9, 0.999), eps=1e-8)          # Оптимизатор
DATASIZE = 10                                                      # Размер датасетов?
TSTEP = 1                                                          # Шаг времени для датасетов? или типа их количество?
N_EXP_TRAIN = 6                                                    # Размер данных для обучения
N_EXP_TEST = 2                                                     # Размер даных для теста
N_EXP = N_EXP_TRAIN + N_EXP_TEST                                   # Общий размер данных
NOISE = 5e-2                                                       # ШУМ
NS = 5                                                             # Количество веществ
NR = 4                                                             # Количество хим. реакций
K = torch.tensor([0.1, 0.2, 0.13, 0.3], dtype=torch.float32)                                   # константы хим. реакций
ATOL = 0.00001                                                     # Параметр точности для ОДУ
RTOL = 0.01                                                        # Параметр точности для ОДУ

MAXITERS = 10000                                                   # Не понял для чего нужно

LB = 0.00001
UB = 1


In [ ]:
# u0_list = np.array([
#     [0.2, 0.4, 0.0, 0.0, 0.0],
#     [1.0, 0.7, 0.0, 0.0, 0.0],
#     [0.8, 0.2, 0.0, 0.0, 0.0],
#     [0.4, 0.2, 0.0, 0.0, 0.0],
#     [0.4, 0.7, 0.0, 0.0, 0.0],
#     [0.4, 0.6, 0.0, 0.0, 0.0],
#     [0.7, 0.2, 0.0, 0.0, 0.0],
#     [0.6, 1.1, 0.0, 0.0, 0.0]
# ])

u0_list = np.random.rand(N_EXP, NS).astype(np.float32)
u0_list[:, 0:2] += 0.2
u0_list[:, 2:] = 0.0

tspan = [0.0, DATASIZE * TSTEP]
tsteps = np.linspace(tspan[0], tspan[1], DATASIZE)               # возможно надо оставить просто DATASIZE
ode_data_list = np.zeros((N_EXP, NS, DATASIZE), dtype=np.float32)


std_list = []

In [6]:
def true_ode_func(t, y, k):
    """
    Вычисляет производные системы дифференциальных уравнений.
    
    Параметры:
    t : float
        Время (не используется в данном уравнении, но необходим для совместимости с solve_ivp).
    y : array-like
        Вектор переменных (y[0], y[1], ..., y[4]).
    k : array-like
        Коэффициенты (k[0], k[1], ..., k[4]).

    Возвращает:
    dydt : numpy.ndarray
        Вектор производных.
    """
    dydt = np.zeros_like(y)
    dydt[0] = -2 * k[0] * y[0]**2 - k[1] * y[0]
    dydt[1] = k[0] * y[0]**2 - k[3] * y[1] * y[3]
    dydt[2] = k[1] * y[0] - k[2] * y[2]
    dydt[3] = k[2] * y[2] - k[3] * y[1] * y[3]
    dydt[4] = k[3] * y[1] * y[3]
    return dydt


In [ ]:
std_list = []

for i in range(N_EXP):
    u0 = u0_list[i, :]  # Начальные условия
    # Решение
    sol = solve_ivp(
        true_ode_func,              # Функция ОДУ
        tspan,                      # Интервал времени
        u0,                         # Начальные условия
        args=(K,),                  # Передача параметров в функцию
        method='RK45',              # Метод решения
        t_eval=tsteps               # Точки, в которых сохраняется решение (Шаг)
    )
    # Преобразование решения
    ode_data = sol.y.T  # Транспонируем для согласования с форматом (в Julia это Array)
    ode_data_list[:, i, :] = ode_data
    std_list.append(np.max(ode_data, axis=1) - np.min(ode_data, axis=1) + LB)      # С добавлением шума
    # std_list.append(np.max(ode_data, axis=0) - np.min(ode_data, axis=0))         # Без добавления шума

# Вычисляем y_std
std_matrix = np.column_stack(std_list)  # Аналог hcat в Julia
y_std = np.max(std_matrix, axis=1)

## Всё что выше, работает!

In [24]:
print(std_list[0])
display(std_list)

display(y_std)
# print()
# print(ode_data_list)
# print()
# print(y_std)

[0.35977 0.0684  0.15625 0.08478 0.08152]


[array([0.35977, 0.0684 , 0.15625, 0.08478, 0.08152]),
 array([1.12025, 0.20305, 0.38046, 0.14806, 0.26251]),
 array([0.92738, 0.11507, 0.33214, 0.20461, 0.15263]),
 array([0.54651, 0.07278, 0.22106, 0.14367, 0.09245]),
 array([0.54653, 0.11936, 0.22104, 0.09127, 0.14484]),
 array([0.54652, 0.11191, 0.22104, 0.09921, 0.1369 ]),
 array([0.83146, 0.10431, 0.30632, 0.19163, 0.13729]),
 array([0.73598, 0.17466, 0.2793 , 0.08452, 0.21487])]

array([1.12025, 0.20305, 0.38046, 0.20461, 0.26251])